In [ ]:
import os
os.chdir('..')

In [ ]:
from torchvision.datasets import EMNIST

In [ ]:
digits_train_ds = EMNIST(root="../data", split='byclass', train=True, download=True)

In [ ]:
digits_train_ds, next(next(iter(digits_train_ds)))

In [ ]:
digits_train_ds.class_to_idx

In [ ]:
import tensorflow_federated as tff
from pathlib import Path
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
tf_ds = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[0])

In [ ]:
lens = [len(list(emnist_train.create_tf_dataset_for_client(client_id).as_numpy_iterator())) for client_id in emnist_train.client_ids]

In [ ]:
sum(lens)

In [ ]:
lens_test = [len(list(emnist_test.create_tf_dataset_for_client(client_id).as_numpy_iterator())) for client_id in emnist_test.client_ids]

In [ ]:
sum(lens_test)

In [ ]:
def process_emnist_tff_set(emnist, base_dir):
    try:
        os.makedirs(base_dir)
    except OSError:
        return
    
    for client_id in tqdm(emnist.client_ids):
        tf_ds = emnist.create_tf_dataset_for_client(client_id)
        x = [np.expand_dims(x['pixels'], axis=0) for x in tf_ds.as_numpy_iterator()]
        x = np.stack(x)
        y = [np.expand_dims(x['label'], axis=0) for x in tf_ds.as_numpy_iterator()]
        y = np.stack(y)
        np.save(base_dir / client_id, {'x': x, 'y': y})

In [ ]:
base_dir = Path('data')
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
process_emnist_tff_set(emnist_train, base_dir / 'train')
process_emnist_tff_set(emnist_test, base_dir / 'test')

In [ ]:
try:
    os.makedirs(base_dir / 'train', exist_ok=False)
except OSError:
    print('it is here')